# Example of Creating Aligned Hierarchies for a Mazurka Score

In this example, we will walk through the elements of the `repytah` package that pertain to the creation of aligned hierarchies for a music-based data stream (eg. a song).

Beginning with features (such as MFCCs and chroma features) for each timestep in your music based data stream, there are several steps to this process:

0. Create the self-dissimilarity matrix (SDM).
1. Highlight pairs of timesteps that are close enough to be considered as repetitions of each other. (In other words, threshold the SDM)
2. Find pairs of structure repetitions (represented as diagonals within the thresholded SDM).
3. Find any pairs of structure repetitions not found in step 2, and group the structure repetitions.
4. Remove any repeated structures that have overlapped instances.
5. Distill the collection of repeated structures into the _essential structure components_ , i.e. the smallest meaningful repetitions on which all larger repeats are constructed. Each timestep will be contained in no more than one essential structure component

_Note_ : This python notebook borrows heavily from `example.py` found in this directory. 


An example function pipeline is shown below to illustrate the function calls of each step. For an in-depth look at each function, check the corresponding jupyter notebooks for each module. 

- <font color='orange'>Yellow: utilities</font>
- <font color='mediumslateblue'>Purple: search</font>
- <font color='olivedrab'>Green: transform</font>
- <font color='indianred'>Red: assemble</font>

<img src="pictures/function_pipeline.jpg" width="380">

We begin by importing the necessary packages: 

In [ ]:
# NumPy and SciPy are required for mathematical operations
import scipy.io as sio
import numpy as np

# Pandas is used to import the csv
import pandas as pd

# utilities imports needed functions
import sys, os
current_dir = os.path.dirname(os.path.realpath('__file__'))
relative_path = os.path.join(current_dir, '..')
sys.path.append(relative_path)
from repytah.utilities import *
from repytah.search import *
from repytah.transform import *
from repytah.assemble import *

# Matplotlib is used to display outputs
import matplotlib.pyplot as plt

## Step 0: Creating the SDM 

In this phase, there are a few crucial details, namely importing the data file that we would like to unearth hierarchical structural information for and determining the appropriate dissimilarity measure to use. If you already have (symmetrical) matrix representations for your data stream, then you may find it more appropriate to load your matrix and then skip ahead to Step 1 or Step 2. 

This step assumes that your music-based data stream (ie. a recording or score) has already had your preferred features extracted (like chroma or MFCCs) and is arranged into columns such that each column represents a time step (or beat). We refer to this as a _feature vector matrix_ as each feature vector is laid out as column within one cohesive matrix. 

### Importing Data for structure analysis

Here we import the file in a familiar manner:

In [ ]:
# Import csv
file_in = pd.read_csv(os.path.join(os.path.dirname('__file__'), "../input.csv")).to_numpy()

fv_mat = file_in

In [ ]:
# Number of feature vectors per shingle
num_fv_per_shingle = 3

# Create the self-dissimilarity matrix
self_dissim_mat = create_sdm(fv_mat, num_fv_per_shingle)

print('self_dissim_mat:\n', self_dissim_mat)

# Produce a visualization
SDM = plt.imshow(self_dissim_mat, cmap="RdBu")
plt.title('Self-Dissimilarity Matrix')
plt.show()


### Creating the SDM

In just one line, we define the self-dissimilarity matrix. This function `create_sdm`, uses feature vectors to create an audio shingle for each time step and represents these shingles as vectors by stacking the relevant feature vectors on top of each other. Then, the cosine distance is found between these shingles.

## Step 1: Thresholding the SDM 

In this step, the self-dissimilarity matrix is thresholded to produce a binary matrix of the same dimensions. This matrix is used to identify repeated structures, which are represented by diagonals of the same length. 

In [ ]:
song_length = self_dissim_mat.shape[0]
thresh = 0.01

# Threshold the SDM to produce a binary matrix 
thresh_dist_mat = (self_dissim_mat <= thresh) 

print('thresh_dist_mat:\n', thresh_dist_mat)

# Produce a visualization
SDM = plt.imshow(thresh_dist_mat, cmap="Greys")
plt.title('Thresholded Matrix')
plt.show()

## Step 2: Find the diagonals present and store all found pairs of repeats in a list

The diagonals in the thresholded matrix are found and recorded in an array of repeats. `find_initial_repeats` does this by looking for the largest repeated structures in *thresh\_mat*, which are illustrated in the above Thresholded Matrix diagram. Once all repeated structures are found, which are represented as diagonals present in *thresh\_mat*, they are stored with their start/end indices and lengths in a list. As each diagonal is found, it is removed to avoid identifying repeated sub-structures.

In [ ]:
all_lst = find_initial_repeats(thresh_dist_mat, np.arange(1,song_length+1), 0)
    
print('all_lst:\n', all_lst)

## Step 3: Find any diagonals in T not found in step 2 contained in larger diagonals in T, then group pairs of repeats

Any diagonals that are contained in larger diagonals are found and added to the array of repeats. All possible repeat lengths are looped over in *all\_lst*, and larger repeats containing smaller repeats are broken up into up to 3 sections each, the part before the overlap, the overlap, and the part after the overlap. The figure below illustrates how 2 overlapping repeats are broken up into 3 sections. With this, a complete list of overlaps is created. 

![alt text](pictures/find_complete_list_visualization.png)




In [ ]:
complete_lst = find_complete_list(all_lst, song_length)
    
print('complete_lst:\n', complete_lst)

## Step 4: Remove any repeated structure that have at least two repeats that overlap

In this step, repeated structures with the same annotation and length are removed if they are overlapping.

This is done by looping over all possible repeat lengths, finding all the groups of repeats of the same length. For each of those groups, `remove_overlaps` determines whether there exists any pair of repeats that overlaps in time. If a pair like that exists, all the overlapping repeats are removed. 

Along with a list of all the repeats with no overlaps, this step also forms a matrix, a list of the associated lengths of the repeats in the matrix, the annotations of the repeats in the matrix, and a list of the overlaps. The matrix is a binary matrix that visualizes the repeats, representing the start of a repeat with a 1. This matrix, in combination with the list of the lengths of the repeats, can be used to visualize the repeats. 


In [ ]:
output_tuple = remove_overlaps(complete_lst, song_length)

print('List with no overlaps:\n', output_tuple[0])
print('Matrix with no overlaps:\n', output_tuple[1])
print('Lengths of the repeats in the matrix:', output_tuple[2])
print('Annotations of the repeats in the matrix:', output_tuple[3])
print('List of overlaps:\n', output_tuple[4])

## Step 5: Find the essential structure components of the song and build the aligned hierarchies

`hierarchical_structure` first finds the essential structure components with `breakup_overlaps_by_intersect`. After this, modified versions of steps 2-4 take place to create the full hierarchical structure. 

After creating the essential structure components, a diagonal matrix of the starting indices of the essential structure components is made, from which information about the combinations of essential structures is extracted following steps similar to those of 2 through 4. From the diagonal matrix, all diagonals are extracted, starting with the longest one, but are not removed. This way, all possible combinations of essential structure components are found. Then, all these combinations are grouped and checked for overlapping repeated combinations. The final product is then restructured to show all the widths and annotations.


In [ ]:
(mat_no_overlaps, key_no_overlaps) = output_tuple[1:3]

# Distill non-overlapping repeats into essential structure components and
# use them to build the hierarchical representation
output_tuple = hierarchical_structure(mat_no_overlaps, key_no_overlaps, song_length, vis=True)
